<a href="https://colab.research.google.com/github/czanalytics/czanalytics/blob/main/logic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logic Programming

We explore ways to use symbolic reasoning using Knowldege Base
(facts, rules, targets) for logic reasoning.

In addition to basic introduction, we discuss possible use of KB in

- machine learning
- industrial use cases
- business rules

Refs.

- https://github.com/MNoorFawi/pytholog
- https://github.com/mnoorfawi/traversing-graphs-using-pytholog


In [ ]:
!pip install pytholog

In [ ]:
import pytholog as pl

In [ ]:
new_kb = pl.KnowledgeBase("flavor")

new_kb(["likes(noor, sausage)",
        "likes(melissa, pasta)",
        "likes(dmitry, cookie)",
        "likes(nikita, sausage)",
        "likes(assel, limonade)",
        "food_type(gouda, cheese)",
        "food_type(ritz, cracker)",
        "food_type(steak, meat)",
        "food_type(sausage, meat)",
        "food_type(limonade, juice)",
        "food_type(cookie, dessert)",
        "flavor(sweet, dessert)",
        "flavor(savory, meat)",
        "flavor(savory, cheese)",
        "flavor(sweet, juice)",
        "food_flavor(X, Y) :- food_type(X, Z), flavor(Y, Z)",
        "dish_to_like(X, Y) :- likes(X, L), food_type(L, T), flavor(F, T), food_flavor(Y, F), neq(L, Y)"])


In [ ]:
new_kb.query(pl.Expr("likes(noor, sausage)")) # ['Yes']
new_kb.query(pl.Expr("likes(noor, pasta)")) # ['No']

In [ ]:
# query 1
from time import time
start = time()
print(new_kb.query(pl.Expr("food_flavor(What, sweet)")))
print(time() - start)

# [{'What': 'limonade'}, {'What': 'cookie'}]
# 0.0020236968994140625

In [ ]:
# query 2
start = time()
print(new_kb.query(pl.Expr("food_flavor(Food, sweet)")))
print(time() - start)

# [{'Food': 'limonade'}, {'Food': 'cookie'}]
# 0.0

In [ ]:
start = time()
print(new_kb.query(pl.Expr("dish_to_like(noor, What)")))
print(time() - start)

# [{'What': 'gouda'}, {'What': 'steak'}]
# 0.001992940902709961


In [ ]:
start = time()
print(new_kb.query(pl.Expr("dish_to_like(noor, What)")))
print(time() - start)

# [{'What': 'gouda'}, {'What': 'steak'}]
# 0.0

In [ ]:
## new knowledge base object
city_color = pl.KnowledgeBase("city_color")
city_color([
    "different(red, green)",
    "different(red, blue)",
    "different(green, red)",
    "different(green, blue)",
    "different(blue, red)",
    "different(blue, green)",
    "coloring(A, M, G, T, F) :- different(M, T),different(M, A),different(A, T),different(A, M),different(A, G),different(A, F),different(G, F),different(G, T)"
])

In [ ]:
## we will use [0] to return only one answer
## as prolog will give all possible combinations and answers
city_color.query(pl.Expr("coloring(Alabama, Mississippi, Georgia, Tennessee, Florida)"), cut = True)

# {'Alabama': 'blue',
#  'Mississippi': 'red',
#  'Georgia': 'red',
#  'Tennessee': 'green',
#  'Florida': 'green'}

In [ ]:
battery_kb = pl.KnowledgeBase("battery")
battery_kb([
	"battery(dead,P) :- voltmeter(battery_terminals,abnormal,P2), P is P2 + 0.5",
	"battery(dead,P) :- electrical_problem(P), P >= 0.8",
	"battery(dead,P) :- electrical_problem(P2), age(battery,old,P3), P is P2 * P3 * 0.9",
	"electrical_problem(0.7)",
	"age(battery,old, 0.8)",
	"voltmeter(battery_terminals,abnormal,0.3)"])

battery_kb.query(pl.Expr("battery(dead, Probability)"))

# [{'Probability': 0.8}, {'Probability': 'No'}, {'Probability': 0.504}]
# the second one is "No" because the condition has not been met.

In [ ]:
iris_kb = pl.KnowledgeBase("iris")
iris_kb([## Rules
	"species(setosa, Truth) :- petal_width(W), Truth is W <= 0.80",
	"species(versicolor, Truth) :- petal_width(W), petal_length(L), Truth is W > 0.80 and L <= 4.95",
	"species(virginica, Truth) :- petal_width(W), petal_length(L), Truth is W > 0.80 and L > 4.95",
	## New record
	"petal_length(5.1)",
	"petal_width(2.4)"])

In [ ]:
iris_kb.query(pl.Expr("species(Class, Truth)"))

# [{'Class': 'setosa', 'Truth': 'No'},
#  {'Class': 'versicolor', 'Truth': 'No'},
#  {'Class': 'virginica', 'Truth': 'Yes'}]

In [ ]:
iris_kb.query(pl.Expr("species(Class, Truth)"))

# [{'Class': 'setosa', 'Truth': 'No'},
#  {'Class': 'versicolor', 'Truth': 'No'},
#  {'Class': 'virginica', 'Truth': 'Yes'}]

In [ ]:
iris_kb.rule_search(pl.Expr("species(Species, Truth)"))

# [species(setosa,Truth):-petal_width(W),TruthisW<=0.80,
#  species(versicolor,Truth):-petal_width(W),petal_length(L),TruthisW>0.80andL<=4.95,
#  species(virginica,Truth):-petal_width(W),petal_length(L),TruthisW>0.80andL>4.95]

In [ ]:
new_kb.clear_cache()

In [ ]:
import pandas as pd
df = pd.DataFrame({"has_work": ["david", "daniel"], "tasks": [8, 3]})
df
#	has_work  tasks
#0	   david	  8
#1	  daniel	  3
ex = pl.KnowledgeBase()
for i in range(df.shape[0]):
    ex([f"has_work({df.has_work[i]}, {df.tasks[i]})"])

ex.db
# {'has_work': {'facts': [has_work(david,8), has_work(daniel,3)],
#   'goals': [[], []],
#   'terms': [['david', '8'], ['daniel', '3']]}}

In [ ]:
graph = pl.KnowledgeBase("graph")
graph([
	"edge(a, b, 6)", "edge(a, c, 1)", "edge(b, e, 4)",
	"edge(b, f, 3)", "edge(c, d, 3)", "edge(d, e, 8)",
	"edge(e, f, 2)",
	"path(X, Y, W) :- edge(X , Y, W)",
	"path(X, Y, W) :- edge(X, Z, W1), path(Z, Y, W2), W is W1 + W2"])

answer, path = graph.query(pl.Expr("path(a, f, W)"), show_path = True)
print(answer)
print([x for x in path if str(x) > "Z"])

# [{'W': 9}, {'W': 12}, {'W': 14}]
# ['d', 'b', 'e', 'c']

In [ ]:
answer, path = graph.query(pl.Expr("path(a, e, W)"), show_path = True, cut = True)
print(answer)
print([x for x in path if str(x) > "Z"])

# [{'W': 10}]
# ['b']

In [ ]:
friends_kb = pl.KnowledgeBase("friends")
friends_kb([
    "stress(X, P) :- has_lot_work(X, P2), P is P2 * 0.2",
    "to_smoke(X, Prob) :- stress(X, P1), friends(Y, X), influences(Y, X, P2), smokes(Y), Prob is P1 * P2",
    "to_have_asthma(X, 0.4) :- smokes(X)",
    "to_have_asthma(X, Prob) :- to_smoke(X, P2), Prob is P2 * 0.25",
    "friends(X, Y) :- friend(X, Y)",
    "friends(X, Y) :- friend(Y, X)",
    "influences(X, Y, 0.6) :- friends(X, Y)",
    "friend(peter, david)",
    "friend(peter, rebecca)",
    "friend(daniel, rebecca)",
    "smokes(peter)",
    "smokes(rebecca)",
    "has_lot_work(daniel, 0.8)",
    "has_lot_work(david, 0.3)"
])

In [ ]:
friends_kb.query(pl.Expr("influences(X, rebecca, P)"))
# [{'X': 'peter', 'P': '0.6'}, {'X': 'daniel', 'P': '0.6'}]

friends_kb.query(pl.Expr("smokes(Who)"))
# [{'Who': 'peter'}, {'Who': 'rebecca'}]

friends_kb.query(pl.Expr("to_smoke(Who, P)"))
# [{'Who': 'daniel', 'P': 0.09600000000000002}, {'Who': 'david', 'P': 0.036}]

friends_kb.query(pl.Expr("to_have_asthma(Who, P)"))
# [{'Who': 'peter', 'P': '0.4'},
#  {'Who': 'rebecca', 'P': '0.4'},
#  {'Who': 'daniel', 'P': 0.024000000000000004},
#  {'Who': 'david', 'P': 0.009}]

In [ ]:
# negation discussion https://stackoverflow.com/questions/70705978/using-not-in-pytholog
# pl has limited syntax, for example not() is missing
# as a solution facts can be introduced sud that True/False is generalized to propabiliry
med_kb = pl.KnowledgeBase()
med_kb([
  "relieves(aspirin, headache)",
  "relieves(new_med, headache)",

  "aggravate(aspirin, peptic_ulcer, 0.8)", # bad
  "aggravate(new_med, peptic_ulcer, 0.3)", # better

  "aggravate(aspirin, asthma, 0.8)",

  "suffers_from(albert, headache)",
  "precondition(albert, peptic_ulcer)",

  # a way to implement neqation using probability
  "should_take(Person, Drug) :- suffers_from(Person, Symptom), precondition(Person, Pre), relieves(Drug, Symptom), aggravate(Drug, _, Prob), Prob <= 0.5",
  "should_not_take(Person, Drug) :- suffers_from(Person, Symptom), precondition(Person, Pre), relieves(Drug, Symptom), aggravate(Drug, _, Prob), Prob > 0.5"
])

In [ ]:
print(med_kb.query(pl.Expr("relieves(Med, _)")))

In [ ]:
print(med_kb.query(pl.Expr("relieves(_, X)")))

In [ ]:
print(med_kb.query(pl.Expr("aggravate(Med, Cond, P)")))

In [ ]:
# new medicine is suggested
print(med_kb.query(pl.Expr("should_take(albert, Drug)")))

In [ ]:
# aspirin should be avoided
print(med_kb.query(pl.Expr("should_not_take(albert, Drug)")))

## Finding paths in a graph

In [ ]:
# https://github.com/mnoorfawi/traversing-graphs-using-pytholog
#
import pytholog as pl

graph_kb = pl.KnowledgeBase("MSA_graph")
graph_kb([## routes between adjacent cities
    "route(seattle, chicago, 1737)",
    "route(seattle, san_francisco, 678)",
    "route(san_francisco, riverside, 386)",
    "route(san_francisco, los_angeles, 348)",
    "route(los_angeles, riverside, 50)",
    "route(los_angeles, phoenix, 357)",
    "route(riverside, phoenix, 307)",
    "route(riverside, chicago, 1704)",
    "route(phoenix, dallas, 887)",
    "route(phoenix, houston, 1015)",
    "route(dallas, chicago, 805)",
    "route(dallas, atlanta, 721)",
    "route(dallas, houston, 225)",
    "route(houston, atlanta, 702)",
    "route(houston, miami, 968)",
    "route(atlanta, chicago, 588)",
    "route(atlanta, washington, 543)",
    "route(atlanta, miami, 604)",
    "route(miami, washington, 923)",
    "route(chicago, detroit, 238)",
    "route(detroit, boston, 613)",
    "route(detroit, washington, 396)",
    "route(detroit, new_york, 482)",
    "route(boston, new_york, 190)",
    "route(new_york, philadelphia, 81)",
    "route(philadelphia, washington, 123)",
	## define the rules how can we move from one point to another
    "path(X, Y, P) :- route(X, Y, P)",
    "path(X, Y, P) :- route(X, Z, P2), path(Z, Y, P3), P is P2 + P3",
	## to make it undirected (two-way) graph
    #"path(X, Y, P) :- route(Y, X, P)",
    "path(X, Y, P) :- route(Y, Z, P2), path(Z, X, P3), P is P2 + P3"
])

In [ ]:
x, y = graph_kb.query(pl.Expr("path(boston, miami, Weight)"), cut = True, show_path = True) ## cut argument to stop searching when a path is found
print(x)
print([x for x in y if str(x) > "Z"]) ## remove weights in the visited nodes

# [{'Weight': 1317}]
# ['washington', 'new_york', 'philadelphia']

In [ ]:
## the other way
x, y = graph_kb.query(pl.Expr("path(miami, boston, Weight)"), cut = True, show_path = True)
print(x)
[x for x in y if str(x) > "Z"]

# [{'Weight': 1317}]
# ['new_york', 'washington', 'philadelphia']


In [ ]:
x, y = graph_kb.query(pl.Expr("path(seattle, washington, Weight)"), cut = True, show_path = True)
print(x)
[x for x in y if str(x) > "Z"]

# [{'Weight': 2371}]
# ['chicago', 'detroit']

In [ ]:
x, y = graph_kb.query(pl.Expr("path(san_francisco, atlanta, Weight)"), cut = True, show_path = True)
print(x)
[x for x in y if str(x) > "Z"]

# [{'Weight': 2678}]
# ['houston', 'dallas', 'riverside', 'chicago']

In [ ]:
x, y = graph_kb.query(pl.Expr("path(chicago, detroit, Weight)"), cut = True, show_path = True)
print(x)
[x for x in y if str(x) > "Z"]

# [{'Weight': '238'}]
# []

In [ ]:
x, y = graph_kb.query(pl.Expr("path(los_angeles, dallas, Weight)"), cut = True, show_path = True)
print(x)
[x for x in y if str(x) > "Z"]

# [{'Weight': 1244}]
# ['phoenix']

In [ ]:
x, y = graph_kb.query(pl.Expr("path(riverside, washington, Weight)"), cut = True, show_path = True)
print(x)
[x for x in y if str(x) > "Z"]

# [{'Weight': 2338}]
# ['miami', 'chicago', 'atlanta', 'detroit']